In [1]:
import numpy as np
import random

In [2]:
data=np.load(r'train.npz')
image, label= data['image'], data['label']
#image_data, label_data = data['image'][..., None], data['label'].astype('int64')
train_x = np.zeros((len(image),28*28),dtype='int64')
train_y = np.zeros((len(label),10),dtype='int64')

for i in range(len(image)):
    train_y[i,int(label[i])]=1
    for j in range(28):
        for k in range(28):
            train_x[i,k+28*j]=int(image[i,j,k])
train_x = train_x.reshape(len(image), -1, 1)
train_y = train_y.reshape(len(label), -1, 1)
print(train_x.shape,train_y.shape)
training_data = list(zip(train_x, train_y))
data=np.load(r'test.npz')
image, label= data['image'], data['label']
test_x = np.zeros((len(image),28*28),dtype='int64')
test_y = np.zeros((len(label),10),dtype='int64')

for i in range(len(image)):
    test_y[i,int(label[i])]=1
    for j in range(28):
        for k in range(28):
            test_x[i,k+28*j]=int(image[i,j,k])
test_x = test_x.reshape(len(image), -1, 1)
test_y = test_y.reshape(len(label), -1, 1)
print(test_x.shape,test_y.shape)
testing_data = list(zip(test_x, test_y))

(12000, 784, 1) (12000, 10, 1)
(5768, 784, 1) (5768, 10, 1)


In [3]:
neurons=[128,64,32,2,10]
for i,j in zip(neurons[1:],neurons[:-1]):
    print(i,j)
for i, j in zip(neurons[:-1], neurons[1:]):
    print(i,j)
for x, y in zip(neurons[:-1], neurons[1:]):
    print(y,x)
#[np.random.randn(y, x) for x, y in zip(neurons[:-1], neurons[1:])]
print(np.random.randn(1,2))
print(type(np.array(neurons)))
print(np.exp(1))

64 128
32 64
2 32
10 2
128 64
64 32
32 2
2 10
64 128
32 64
2 32
10 2
[[1.73392602 1.13440008]]
<class 'numpy.ndarray'>
2.718281828459045


In [4]:
print(1-np.array([0.7,0.6]))

[0.3 0.4]


In [5]:
def softmax(z):
    z = np.log(np.maximum(z, 1e-9))
    exps = np.exp(z)    # -max(z).all()
    sums = np.sum(exps) # -max(z).all()
    return np.divide(exps, sums)
def sigmoid(z):
    for (x, y), val in np.ndenumerate(z):
        if val >= 100:
            z[x][y] = 1.
        elif val <= -100:
            z[x][y] = 0.
        else:
            z[x][y] = 1.0 / (1.0 + np.exp(-val))
    return z
def softmax_derivate(z):
    z=z.flatten()
    jacobian_m = np.diag(z)
    for i in range(len(jacobian_m)):
        for j in range(len(jacobian_m)):
            if i == j:
                #print(i,j)
                #print(z.shape)
                jacobian_m[i][j] = z[i] * (1-z[i])
            else: 
                jacobian_m[i][j] = -z[i]*z[j]
    return jacobian_m
#def sigmoid(z):
#    return 1.0 / (1.0 + np.exp(-z)) 不設限制很容易爆炸
def sigmoid_derivate(z):
    return sigmoid(z) * (1-sigmoid(z))
def relu(z):
    return np.maximum(0,z)
def relu_derivative(z):
    return np.where(z <= 0, 0, 1)
def cross_entropy(output,ground_truth):
    output = np.clip(output, 1e-9, 1. - 1e-9)
    ce = -(np.sum(np.nan_to_num(ground_truth*np.log(output)))) /len(output)
    return ce
#def bin_cross_entropy(output, ground_truth):
#    return np.sum( np.nan_to_num( -ground_truth*(np.log(output)) - (1-ground_truth)*(np.log(1-output)) ) )
def cross_entropy_derivative(output, ground_truth): #softmax and cross_entropy derivate
    return output - ground_truth

In [6]:
class DNN():
    def __init__(self,neurons,rd=False):
        self.num_layers=len(neurons)
        self.neurons=neurons
        if rd==True:
            self.random_initial()
        else:
            self.zero_initial()
        self.training_loss = []
        self.training_error_rate = []
        self.testing_error_rate = []
    def random_initial(self):
        self.weights =[np.random.randn(i,j)for i,j in zip(neurons[1:],neurons[:-1])] #random_normal number =len(neurons-1)
        self.biases  =[np.random.randn(j,1)for j in neurons[1:]] #add after data mutiby weight so (m,1)
        print("random_initial")
    def zero_initial(self):
        self.weights =[np.zeros((i,j),dtype='float')for i,j in zip(neurons[1:],neurons[:-1])] # transposed (n,m)->(m,n)
        self.biases  =[np.zeros((j,1),dtype='float')for j in neurons[1:]] #add after data mutiby weight so (m,1)
        print("zero_initial")
    def SGD(self,train_data,test_data,epochs,batch_size,lr): #randomly choose data as batch size
        num = len(train_data)
        for i in range(epochs):
            random.shuffle(train_data)
            mini_batch = [train_data[j:j+batch_size] for j in range(0,num,batch_size) ]
            for data in mini_batch:
                self.update_mini_batch(data,lr)
            if (i==20):
                w_20 = self.weights
                b_20 = self.biases
            if (i==80):
                w_80 = self.weights
                b_80 = self.biases
            if (i % 50 == 0):
                # record info
                self.training_loss.append(self.calc_loss(training_data))
                self.training_error_rate.append(self.count_error(training_data) / len(training_data))
                self.testing_error_rate.append(self.count_error(testing_data) / len(testing_data))
                print('===================================')
                print("【Epoch %s】" % i) 
                print('    training loss: %f' % self.calc_loss(training_data))
                print('    training error rate: %d / %d(%f)' % (self.count_error(training_data), len(training_data), self.count_error(training_data) / len(training_data)))
                print('    testing error rate: %d / %d(%f)' % (self.count_error(testing_data), len(testing_data), self.count_error(testing_data) / len(testing_data)))

    def update_mini_batch(self, batch_data, lr):
        sum_gradient_w = [ np.zeros(w.shape) for w in self.weights ] #creat same size as weight think as 暫存器
        sum_gradient_b = [ np.zeros(b.shape) for b in self.biases ]
        # cumulate gradient of each single data
        for x, y in batch_data:
            gradient_w, gradient_b = self.backward(x, y)
            sum_gradient_w = [ sw + w for sw, w in zip(sum_gradient_w, gradient_w)] #sum up to update 
            sum_gradient_b = [ sb + b for sb, b in zip(sum_gradient_b, gradient_b)] #gradient afer each batch
        # update weights & biases with (mean of sum of gradient * learning rate)
        #print(sum_gradient_w,sum_gradient_b)
        self.weights = [ w - lr/len(batch_data) * sw for w, sw in zip(self.weights, sum_gradient_w) ]# weight-average_weight*lr
        self.biases = [ b - lr/len(batch_data) * sb for b, sb in zip(self.biases, sum_gradient_b) ]

    def forward(self, x):
        z=[] # use to derivation
        activation_z=[x]
        for layer in range(self.num_layers-1):
            if layer ==(self.num_layers-2):
                #print(self.weights[4].shape,self.biases[4].shape)
                x = np.dot(self.weights[layer],x)+self.biases[layer]
                z.append(x)
                x=softmax(x)
                activation_z.append(x)
            else:
                x = np.dot(self.weights[layer],x)+self.biases[layer]
                z.append(x)
                x=sigmoid(x)
                activation_z.append(x)
        return x,z,activation_z
    def backward(self,x,y):
        #create tmp weight and bias
        gradient_w = [ np.zeros(w.shape) for w in self.weights ] #0~len(neouron)-2
        gradient_b = [ np.zeros(b.shape) for b in self.biases ]
        # forward
        _,zs,activation_z=self.forward(x)
        # backward 
        # we calc last layer separately, from back to forward
        for layer in range(1,self.num_layers):
            if layer == 1: #first initial
                delta = cross_entropy_derivative(activation_z[-layer],y)
                #delta = cross_entropy_derivative(activation_z[-layer],y) * sigmoid_derivate(zs[-1])
                #delta = softmax_derivate(z[-1])
                gradient_b[-layer]=delta * 1
                gradient_w[-layer]=np.dot(delta,activation_z[-layer-1].T)
            else:
                delta = np.dot(self.weights[-layer + 1].T, delta) * sigmoid_derivate(zs[-layer])
                #delta = np.dot(self.weights[-layer + 1].T, delta) * relu_derivate(zs[-layer])
                gradient_w[-layer] = np.dot(delta, activation_z[-layer - 1].T)
                gradient_b[-layer] = delta
        return gradient_w, gradient_b
    def calc_loss(self,data):
    # calc cross entropy loss
        loss = 0
        for x, y in data:
            output,_,_ = self.forward(x)
            #output = self.forward(x)
            #print("Output:{},Ground Truth:{}".format(output,y))
            loss += cross_entropy(output, y) #/ len(data)
        return loss
    def count_error(self, data):
    # count error number
        compare_list=[]
        for x , y in data:
            #x = self.forward(x)
            x,_,_ = self.forward(x)
            x = x.argmax()
            y = np.argmax(y)
            #print(x,y)
            compare_list.append([x,y])
        #compare_list = [ (np.argmax(self.forward(x)), np.argmax(y)) for x, y in data ]
        error_count = sum( int(y1 != y2) for y1, y2 in compare_list)
        return error_count 



In [7]:
def forward(self, x):
        z=[] # use to derivation
        activation_z=[x]
        for layer in range(self.num_layers-1):
            if layer ==(self.num_layers-2):
                #print(self.weights[4].shape,self.biases[4].shape)
                x = np.dot(self.weights[layer],x)+self.biases[layer]
                z.append(x)
                x=softmax(x)
                activation_z.append(x)
            else:
                x = np.dot(self.weights[layer],x)+self.biases[layer]
                z.append(x)
                x=sigmoid(x)
                activation_z.append(x)
        return x,z,activation_z
#    def forward(self,x):
#        for w,b in zip(self.weights,self.biases):
#            x = np.dot(w,x)+b
#            x = sigmoid(x)
#        return x
        activation = x
        activation_z = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activation_z.append(activation)

In [8]:
class NN():
    def __init__(self, neurons):
        self.num_layers = len(neurons)
        self.neurons = neurons
        # create weights & bias
        self.weights = [ np.zeros((j, i)) for i, j in zip(neurons[:-1], neurons[1:]) ]
        self.biases = [ np.zeros((i, 1)) for i in neurons[1:] ]
        # info 
        self.training_loss = []
        self.training_error_rate = []
        self.testing_error_rate = []
    
    def SGD(self, training_data, testing_data, epochs, batch_size, lr):

        num = len(training_data)
        self.training_loss = []
        self.training_error_rate = []
        self.testing_error_rate = []
        
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        
        for epoch in range(epochs):
            random.shuffle(training_data)
            mini_batch = [ training_data[i : i + batch_size] for i in range(0, num, batch_size) ] # split data into mini_batch
            for single_data in mini_batch:
                self.update_mini_batch(single_data, lr)
            if (epoch == 20):
                weight_20 = self.weights
                bias_20 = =self.biases
            if (epoch == 80):
                weight_80 = self.weights
                bias_80 = self.biases
            if (epoch % 50 == 0):
                # record info
                self.training_loss.append(self.calc_loss(training_data))
                self.training_error_rate.append(self.count_error(training_data) / len(training_data))
                self.testing_error_rate.append(self.count_error(testing_data) / len(testing_data))
                print('===================================')
                print("【Epoch %s】" % epoch) 
                print('    training loss: %f' % self.calc_loss(training_data))
                print('    training error rate: %d / %d(%f)' % (self.count_error(training_data), len(training_data), self.count_error(training_data) / len(training_data)))
                print('    testing error rate: %d / %d(%f)' % (self.count_error(testing_data), len(testing_data), self.count_error(testing_data) / len(testing_data)))
        return weight_20,bias_20,weight_80,bias_80
                
    
    def update_mini_batch(self, single_data, lr):
        sum_gradient_w = [ np.zeros(w.shape) for w in self.weights ]
        sum_gradient_b = [ np.zeros(b.shape) for b in self.biases ]
        
        # cumulate gradient of each single data
        for x, y in single_data:
            gradient_w, gradient_b = self.backward(x, y)
            sum_gradient_w = [  sw + w for sw, w in zip(sum_gradient_w, gradient_w)]
            sum_gradient_b = [ sb + b for sb, b in zip(sum_gradient_b, gradient_b)]
        
        # update weights & biases with (mean of sum of gradient * learning rate)
        self.weights = [ w - lr/len(single_data) * sw for w, sw in zip(self.weights, sum_gradient_w) ]
        self.biases = [ b - lr/len(single_data) * sb for b, sb in zip(self.biases, sum_gradient_b) ]
    
    def forward(self, x):
        for w, b in zip(self.weights, self.biases):
            x = np.dot(w, x) + b
            x = sigmoid(x)
        return x
    
    def backward(self, x, y):
        # store gradient of w, b
        gradient_w = [ np.zeros(w.shape) for w in self.weights ]
        gradient_b = [ np.zeros(b.shape) for b in self.biases ]
        
        # forward
        activation = x
        zs = [] # store vectors which is input of activation function
        activations = [x] # store vectors which is output of activation function
        
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
            
        # backward 
        # we calc last layer separately, because loss function is diff with activation funcion
        delta = cross_entropy_derivative(activations[-1], y)
        gradient_b[-1] = delta * 1
        gradient_w[-1] = np.dot(delta, activations[-2].T)
        for layer in range(2, self.num_layers):
            z = zs[-layer]
            delta = np.dot(self.weights[-layer + 1].T, delta) * sigmoid_derivate(z)
            gradient_w[-layer] = np.dot(delta, activations[-layer - 1].T)
            gradient_b[-layer] = delta
        return gradient_w, gradient_b
        
    def calc_loss(self, data):
        # calc cross entropy loss
        loss = 0
        for x, y in data:
            output = self.forward(x)
            loss += cross_entropy(output, y)/ len(data)
        return loss
    
    def count_error(self, data):
        # count error number
        compare_list = [ (np.argmax(self.forward(x)), np.argmax(y)) for x, y in data ]
        error_count = sum( int(y1 != y2) for y1, y2 in compare_list)
        return error_count

SyntaxError: invalid syntax (<ipython-input-8-9320ba7591b7>, line 30)

In [ ]:
neurons=[784,128,2,10]
#neurons=[784,100,10]

#module1 = DNN(neurons,True)
module1 = NN(neurons)
w_2,b_2,w_8,b_8=module1.SGD(training_data, testing_data, 100, 60, 0.08)